# [Solved] Lab 1: **Kernels** and **features**

Advanced Topics in Machine Learning -- Spring 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/ganselmif/adv-ml-units/blob/main/solutions/AdvML_UniTS_2023_Lab_01_Intro_to_Kernels_Solved.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

### Empirical verification of the *Kernel* $\leftrightarrow$ *feature expansion* equivalence

Recall the definition of *kernel*:

> Let $\mathcal{X}$ be a non-empty set. A function $k: \mathcal{X} \times \mathcal{X} \rightarrow \mathbb{R}$ is called a *kernel* if there exists a real-Hilbert space $\mathcal{H}$ and a map $\phi: \mathcal{X} \rightarrow \mathcal{H}$ such that $\forall x, x^\prime \in \mathcal{X}$, $k(x, x^\prime) := \langle \phi(x) , \phi(x^\prime) \rangle_{\mathcal{H}}$.

To motivate the usefulness of kernelized ML methods, we can show that -- for $x\in\mathbb{R}^{d \in \mathbb{N}}$ -- the computation of $k(x, x^\prime)$ in kernel form is equivalent to the explicit scalar product $\langle \varphi(x) , \varphi(x^\prime) \rangle = \varphi(x)^{T} \varphi(x^\prime)$ of some corresponding expanded feature maps $\varphi: {R}^{d} \rightarrow \mathbb{R}^{d^\prime}$ with generally $d^\prime \gg d$ (or even *infinite-dimensional* $\varphi$s), though significantly simpler and more efficient to compute.

In the lab that follows, verify such equivalence for simple kernels: the non-uniform *quadratic* (in $\mathbb{R}^{d}$) and the *Gaussian* (in $\mathbb{R}$).

For each kernel:

1. Implement a function that computes the kernel between two arrays of coordinates;
2. Derive the explicit feature map $\varphi(x)$ corresponding to that kernel;
3. Implement a function that computes such feature map for a given array of coordinates;
4. Verify that the kernel computed by (1) and the scalar product of its arguments through (3) are indeed equivalent.

**Hint**: in case of need, you can finitely approximate the feature map by Taylor expansion.

In [29]:
import itertools
import math
import numpy as np

In [30]:
def nu_quadratic_kernel(x, x_prime):
    """Compute the quadratic kernel between two arrays of coordinates.

    Parameters
    ----------
    x : array-like, shape (n_features)
        First array of coordinates.
    x_prime : array-like, shape (n_features)
        Second array of coordinates.

    Returns
    -------
    k : array-like, shape (1)
        Kernel value.
    """
    x, x_prime = np.asarray(x), np.asarray(
        x_prime
    )  # Always a good practice; almost overhead-free.
    return (1 + np.dot(x, x_prime)) ** 2

In [31]:
def nu_quadratic_feature_map(x):
    """Compute the feature map corresponding to the quadratic kernel.

    Parameters
    ----------
    x : array-like, shape (n_features)
        Array of coordinates.

    Returns
    -------
    phi_x : array-like, shape (n_features)
        Feature map.
    """

    x = np.asarray(x)

    # Mixed products
    mixed = np.asarray(
        [
            math.sqrt(2) * x[i] * x[j]
            for i, j in itertools.combinations(range(len(x)), 2)
        ]
    )

    return np.concatenate((np.asarray([1]), math.sqrt(2) * x, x**2, mixed))

In [ ]:
# Check the equivalence on randomly-initialized arrays

v = np.random.randn(100)
u = np.random.randn(100)

kernel_val = nu_quadratic_kernel(v, u)
feature_map_val = nu_quadratic_feature_map(v).dot(nu_quadratic_feature_map(u))

if np.allclose(kernel_val, feature_map_val):
    print("Success!")

In [32]:
def gaussian_kernel(x, x_prime, sigma):
    """Compute the Gaussian kernel between two arrays of coordinates.

    Parameters
    ----------
    x : array-like, shape (1)
        First array of coordinates.
    x_prime : array-like, shape (1)
        Second array of coordinates.
    sigma : float
        Kernel standard deviation.

    Returns
    -------
    k : array-like, shape (1)
        Kernel value.
    """
    x, x_prime = np.asarray(x), np.asarray(x_prime)
    return np.exp(-((x - x_prime) ** 2) / (2 * sigma**2))

In [33]:
def gaussian_feature_map(x, sigma, approx_order=100):
    """Compute the feature map corresponding to the Gaussian kernel.

    Parameters
    ----------
    x : array-like, shape (n_features)
        Array of coordinates.
    sigma : float
        Kernel standard deviation.
    approx_order : int, optional (default=100)
        Order of the Taylor expansion used to approximate the feature map.

    Returns
    -------
    phi_x : array-like, shape (n_features)
        Feature map.
    """

    x = np.asarray(x)

    common_factor = np.exp(-(x**2) / (2 * sigma**2))

    taylor = (
        common_factor
        * np.asarray(
            [
                (x / sigma) ** i / math.sqrt(math.factorial(i))
                for i in range(approx_order)
            ]
        ).flatten()
    )
    # Flattening is required as otherwise we would have [[...],[...],[...], ...]

    return taylor

In [36]:
# Check the equivalence on randomly-initialized arrays

v = 100 * np.random.rand(1)
u = 100 * np.random.rand(1)

chosen_sigma = 10

kernel_val = gaussian_kernel(v, u, sigma=chosen_sigma)
feature_map_val = gaussian_feature_map(v, sigma=chosen_sigma).dot(
    gaussian_feature_map(u, sigma=chosen_sigma)
)

if np.allclose(kernel_val, feature_map_val):
    print("Success!")


Success!
